In [ ]:
import os
import glob
import pandas as pd
import geopandas as gpd
import pickle
import param as pm
import panel as pn
pn.extension('echarts')
import numpy as np
import xarray as xr
import rioxarray as riox
import holoviews as hv
import hvplot.xarray
from holoviews import streams
import affine
from bokeh.models.formatters import PrintfTickFormatter
from bokeh.models import NumeralTickFormatter, FixedTicker
from holoviews.plotting.util import process_cmap
from datetime import datetime, timedelta
from pyproj import Proj, transform
from shapely.geometry import Polygon
#from src.hls_funcs import fetch
#from src.hls_funcs.masks import mask_hls, shp2mask, bolton_mask
#from src.hls_funcs.indices import ndvi_func
#from src.hls_funcs.smooth import smooth_xr, despike_ts_xr
#from src.hls_funcs.predict import pred_bm, pred_cov, pred_bm_se, pred_bm_thresh, xr_cdf
from src.hls_funcs.masks import shp2mask
from src.hls_funcs.predict import xr_cdf
#from google.cloud import storage

pn.param.ParamMethod.loading_indicator = True

In [ ]:
#os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/mnt/c/Users/TBGPEA-Sean/OneDrive/Documents/gcloud/range-nrt-505e4c7ce3e5.json"
# Instantiates a client
#client = storage.Client()

#bucket = client.get_bucket('hls_nrt')

#blob = bucket.get_blob('cper/hls_biomass/cper_hls_bm_2022.tif')

In [ ]:
"""
Get the server base url
"""
"""
Get the server base url
"""
from jupyter_server import serverapp
 
try:
    jupServer = [x for x in serverapp.list_running_servers()][0]
except IndexError:
    jupServer = {'base_url': '/node/ceres19-compute-31-eth.scinet.local/35403/'}

In [ ]:
print('   setting up Local cluster...')
#from dask.distributed import LocalCluster, Client
#import dask
#cluster = LocalCluster(n_workers=2, threads_per_worker=2)
#client = Client(cluster)

In [ ]:
cper_f = os.path.join('data/ground/cper_pastures_2017_clip.shp')
prefix = 'cper'

In [ ]:
sngl_chunks = {'date': 1, 'y': -1, 'x': -1}

In [ ]:
yr = 2022
ds = riox.open_rasterio('data/gcloud/hls_' + prefix + '_' + str(yr) + '_gcloud.nc', masked=True)#.chunk(sngl_chunks)
ds['date'] = [datetime.strptime(str(x),'%Y-%m-%d %H:%M:%S') for x in ds['date'].values]
ds['date'] = ds['date'].dt.date
ds = ds.where(ds < ds.attrs['_FillValue'])
ds = ds.where(ds != np.inf)
#ds

In [ ]:
ds

In [ ]:
cper = gpd.read_file(cper_f)
cper_bbox = cper.buffer(2000).to_crs(epsg=3857).total_bounds

In [ ]:
from bokeh.models import WheelZoomTool
wheel_zoom = WheelZoomTool(zoom_on_axis=False)

In [ ]:
class getData(pm.Parameterized):
    control_width = 150
    stats_width = 380
    stats_margin = (5, 1)
    ts_chunks = {'date': -1, 'y': 20, 'x': 20}
    
    viz_sel = pn.widgets.RadioButtonGroup(options=['Basemap', 
                                                   'Cover',
                                                   'Greenness (NDVI)',
                                                   'Biomass', 
                                                   'Biomass probability'],
                                          value='Cover',
                                          align='start', 
                                          orientation='vertical',
                                          width=control_width)

    def viz_callback(target, event):
        if event.new == target.name:
            target.value = True
        else:
            target.value = False

    cov_viz = pn.widgets.Checkbox(name='Cover', value=True)
    bm_viz = pn.widgets.Checkbox(name='Biomass', value=False)
    thresh_viz = pn.widgets.Checkbox(name='Biomass probability', value=False)
    ndvi_viz = pn.widgets.Checkbox(name='Greenness (NDVI)', value=False)

    viz_sel.link(cov_viz, callbacks={'value': viz_callback})
    viz_sel.link(bm_viz, callbacks={'value': viz_callback})
    viz_sel.link(thresh_viz, callbacks={'value': viz_callback})
    viz_sel.link(ndvi_viz, callbacks={'value': viz_callback})
    
    year_picker = pn.widgets.Select(name='Select year', options=list(range(2016, 2023, 1)), value=2022,
                                   width=control_width)
    thresh_picker = pn.widgets.IntSlider(name='Biomass threshold', start=200, end=2000, step=50, value=500,
                                 format=PrintfTickFormatter(format='%d kg/ha'), width=control_width)
    
    data_dict = {'date_range': [str(datetime(year_picker.value, 1, 1).date()),
                                str(datetime(year_picker.value, 12, 31).date())]}
 
    cov_dict = {'R': 'Dry veg',
                'G': 'Green veg',
                'B': 'Bare ground'}

    map_args = dict(rasterize=True, project=False, dynamic=True, xticks=None, yticks=None)
    base_opts = dict(backend='bokeh', xaxis=None, yaxis=None, width=900, height=700,
                         padding=0, active_tools=['pan', wheel_zoom], toolbar='left')
    map_opts = dict(responsive=False, xaxis=None, yaxis=None, width=900, height=700,
                     padding=0, tools=['pan', wheel_zoom, 'box_zoom'],
                     active_tools=['pan', wheel_zoom], toolbar='left')
    
    poly_opts = dict(fill_color=['', ''], fill_alpha=[0.0, 0.0], line_color=['#1b9e77', '#d95f02'],
                 line_width=[3, 3])    
    bg_col='#ffffff'

    css = '''
    .bk.box1 {
      background: #ffffff;
      border-radius: 5px;
      border: 1px black solid;
    }
    '''
    
    date = pn.widgets.DatePicker(name='Select date', width=100)
    enabled_dates = [pd.Timestamp(x).to_pydatetime().date() for x in ds['date'].values]
    date.enabled_dates = enabled_dates
      
    tiles = hv.element.tiles.EsriImagery().opts(**base_opts, level='glyph',
                                                                   xlim=(cper_bbox[0], cper_bbox[2]),
                                                                   ylim=(cper_bbox[1], cper_bbox[3]))
    labels = hv.element.tiles.EsriReference().opts(**base_opts, level='overlay')
    
    base_rng = hv.streams.RangeXY(source=tiles,
                                  x_range = (cper_bbox[0], cper_bbox[2]),
                                  y_range = (cper_bbox[1], cper_bbox[3]))
       
    #basemap = hv.element.tiles.EsriImagery().opts(**map_opts, level='glyph')
        
    polys = hv.Polygons([])

    max_polys = 1
    
    poly_stream = streams.PolyDraw(source=polys, drag=True, num_objects=max_polys,
                                    show_vertices=True, styles=poly_opts)
    edit_stream = streams.PolyEdit(source=polys, shared=True)
    
    bm_cmin = 100
    bm_cmax = 1000
    
    cbar_dict = {
    'Cover': {'cmap': 'brg',
              'clim': (0, 1.0),
              'colorbar_opts': {'height': 20,
                                'width': int(control_width*0.95),
                                'ticker': FixedTicker(ticks=[0.05, 0.50, 0.95]),
                                'major_label_overrides': {0.05: 'Bare', 0.50: 'Dry', 0.95: 'Green'}}},
    'Greenness (NDVI)': {'cmap': 'Viridis',
              'clim': (0.0, 0.5),
              'colorbar_opts': {'height': 20,
                                'width': int(control_width*0.95),
                                'ticker': FixedTicker(ticks=[0.025, 0.175, 0.325, 0.475]),
                                'major_label_overrides': {0.025: 'V. Low', 0.175: 'Low', 0.325: 'Mod', 0.475: 'High'}}},
    'Biomass': {'cmap': 'Inferno',
                'clim': (100, 1500),
                'colorbar_opts': {'height': 20,
                                  'width': int(control_width*0.95),
                                  'ticker': FixedTicker(ticks=np.arange(250, 1750, 250)),
                                  'major_label_overrides': {1500: '1500+'}}},
    'Biomass probability': {'cmap': 'Spectral_r',
              'clim': (0.0, 1.0),
              'colorbar_opts': {'height': 20,
                                'width': int(control_width*0.95),
                                'ticker': FixedTicker(ticks=np.round(np.arange(0.1, 1.1, 0.2), 1)),
                                'major_label_overrides': {0.1: 'Unlikely', 0.3: 'Low',
                                                          0.5: 'Possible', 0.7: 'Likely', 0.9: 'High'}}}
    }
    
    bm_plotrange = np.arange(0, 2500, 50)
    bm_colors = np.array(process_cmap('Inferno', ncolors=len(bm_plotrange)))
    bm_colors[bm_plotrange < bm_cmin] = bm_colors[0]
    bm_colors[bm_plotrange > bm_cmax] = bm_colors[-1]
    bm_colors[(bm_plotrange >= bm_cmin) & 
              (bm_plotrange <= bm_cmax)] = process_cmap('Inferno', ncolors=len(np.arange(bm_cmin, bm_cmax+50, 50)))

    thresh_plotrange = np.arange(0, 1.2, 0.2)
    thresh_colors = process_cmap('Spectral_r', ncolors=len(thresh_plotrange)-1)
    thresh_labels = ['Unlikely', 'Low', 'Possible', 'Likely', 'High']
    
    bm_gauge = pn.indicators.Gauge(
        name='Biomass', bounds=(0, 2500), format='{value} kg/ha', tooltip_format='{b} : {c} kg/ha',
        colors=[(0.20, '#FF6E76'), (0.40, '#FDDD60'), (0.60, '#7CFFB2'), (1, '#58D9F9')],
        num_splits=5, align='center', title_size=12,
        start_angle=180, end_angle=0,
        height=200, width=stats_width,
        value=0, margin=stats_margin)
    
    bm_hist = {
        'title': {
            'text': 'Biomass variability',
            'left': "center"},
        'tooltip': {},
        'legend': {},
        'grid': {'show': False},
        'xAxis': {
            'type': 'value',
            'name': 'Biomass (kg/ha)',
            'nameLocation': 'middle',
            'nameGap': 30,
            'splitLine': {
                'show': False}},
        'yAxis': {'max': 35,
                  'axisLabel': {'formatter': '{value} %'},
                  'splitLine': {
                      'show': False}},
        'series': [{
            'name': "",
            'type': "bar",
            'data': [{'value': [x, 0]} for idx, x in enumerate(
                bm_plotrange)],
            'colorBy': "data"}]}
    bm_echart = pn.pane.ECharts(bm_hist, height=250, width=stats_width, margin=stats_margin)
    
    thresh_bar = {
        'title': {
            'text': "Biomass threshold",
            'left': "center"},
        'tooltip': {},
        'legend': {},
    'grid': {'left': '20%'},
    'xAxis': {
        'type': 'category',
        'name': 'Probability of biomass less than ' + str(thresh_picker.value) + 'kg/ha',
        'nameLocation': 'middle',
        'nameGap': 30,
    'data': thresh_labels},
    'yAxis': {'max': 1000,
              'axisLabel': {'formatter': '{value} ha'}},
    'series': [{
        'name': "",
        'type': "bar",
        'data': [0 for x in thresh_labels],
        'colorBy': "data"
    }],
}
    thresh_echart = pn.pane.ECharts(thresh_bar, height=250, width=stats_width, margin=stats_margin)
    
    cov_pie = {
        'title': {
            'text': "Cover",
            'subtext': "Fractional vegetation cover (%)",
            'left': "center"},
        'tooltip': {'show': True,
                    'formatter': '{b}: {d} %'}, 
        'series': [{
            'type': 'pie',
            'data': [{'value': 0, 'name': 'Litter'},
                     {'value': 0, 'name': 'Bare ground'},
                     {'value': 0, 'name': 'Green veg'},
                     {'value': 0, 'name': 'Dry veg'}],
            'color': ['#ee6666', '#91cc75', '#5470c6', '#fac858'],
            'roseType': 'radius',
            'radius': ["0%", "45%"],
            'label': {
                'edgeDistance': "1%",
                'bleedMargin': 10,
                'alignTo': "edge"},
            'labelLine': {'length': 5}}]}
    
    cov_echart = pn.pane.ECharts(cov_pie, height=250, width=stats_width, margin=stats_margin)
    
    ts_bm = {
        'title': [{'left': 'center', 'text': 'Biomass'}],
        'legend': {'orient': 'vertical',
                                   'right': 10,
                                   'top': 'center'
                                  },
        'tooltip': {'trigger': 'axis'},
        'xAxis': [{'data': ds['date'].astype('str').values}],
        'yAxis': [{}],
        'series': []}    
    ts_bm_echart = pn.pane.ECharts(ts_bm, height=250, width=stats_width, margin=stats_margin)
    
    ts_ndvi = {
        'title': [{'left': 'center', 'text': 'Greenness (NDVI)'}],
        'tooltip': {'trigger': 'axis'},
        'xAxis': [{'data': ds['date'].astype('str').values}],
        'yAxis': [{}],
        'series': []}    
    ts_ndvi_echart = pn.pane.ECharts(ts_ndvi, height=250, width=stats_width, margin=stats_margin)
    
    ts_cov = {
        'title': [{'left': 'center', 'text': 'Cover (fractional)'}],
        'tooltip': {'trigger': 'axis'},
        'xAxis': [{'data': ds['date'].astype('str').values}],
        'yAxis': [{}],
        'series': []}    
    ts_cov_echart = pn.pane.ECharts(ts_cov, height=250, width=stats_width, margin=stats_margin)

    
    def __init__(self, **params):
        super(getData, self).__init__(**params)
        self.ds = ds#.chunk(sngl_chunks)
        self.ds_ts = ds#.chunk(self.ts_chunks)
        self.ds_sel = None

        self.all_maps = None
        self.cov_map = None
        self.bm_map = None
        self.thresh_map = None
        
        self.cov_stats = None
        self.bm_stats = None
        self.thresh_stats = None
        self.stats_titles = None
        self.ts_bm = None
        
        self.date.value = pd.to_datetime(self.ds['date'].values[-1]).date()
                                        
        self.poly_stream.add_subscriber(self.update_stats)
        
        self.map_init = self.tiles * self.labels
        
        self.controls = pn.Column(pn.Spacer(height=30),
                                  self.viz_sel,
                                  pn.Spacer(height=10),
                                  self.thresh_picker,
                                  pn.Spacer(height=20),
                                  pn.pane.Markdown('*Map legend*', background='white'),
                                  self.update_colorbar, height=550, width=int(self.control_width*1.5))
        
        self.view = self._create_view()
    
    def make_colorbar(self, key, orientation = 'horizontal', position = 'top', colorbar_opts = {}, **kwargs):
        ## create an invisible hv.Heatmap plot just to use its colorbar
        cmap = self.cbar_dict[key]['cmap']
        clim = self.cbar_dict[key]['clim']
        colorbar_opts = self.cbar_dict[key]['colorbar_opts']
        
        hm = hv.HeatMap([(0, 0, clim[0]), (0, 1, clim[1])])
        kwargs.update(dict(colorbar=True,
                           colorbar_opts=colorbar_opts, 
                           clim=clim,
                           alpha=0,
                           show_frame=False,
                           frame_height=0,
                           colorbar_position=position, 
                           toolbar="disable",
                           axiswise=True, framewise=True, shared_axes=False,
                           margin=(0,0),
                           cmap=cmap))
        return hm.opts(xaxis = None, yaxis = None, **kwargs)
      
    @pm.depends('date.value', 'thresh_picker.value')
    def create_maps(self):
        self.ds_sel = ds.sel(date=self.date.value).load()
        if self.ds is not None:
            if self.edit_stream.data is not None:
                self.polys = self.edit_stream.element
                self.poly_stream = streams.PolyDraw(source=self.polys, drag=True, num_objects=self.max_polys,
                                                    show_vertices=True, styles=self.poly_opts)
                self.edit_stream = streams.PolyEdit(source=self.polys, shared=True)
            elif self.poly_stream.data is not None:
                self.polys = self.poly_stream.element
                self.poly_stream = streams.PolyDraw(source=self.polys, drag=True, num_objects=self.max_polys,
                               show_vertices=True, styles=self.poly_opts)
                self.edit_stream = streams.PolyEdit(source=self.polys, shared=True)
            else:
                self.polys = self.polys
            self.poly_stream.add_subscriber(self.update_stats)

            da_thresh_pre = (np.log(self.thresh_picker.value) - 
                             np.log(self.ds_sel['Biomass'])) / self.ds_sel['Biomass_SE']
            da_thresh = xr_cdf(da_thresh_pre)
            da_thresh.name = 'Threshold'

            self.cov_map = self.ds_sel[['DRY', 'GREEN', 'BARE']].to_array().hvplot.rgb(x='x', y='y', 
                                                                                       bands='variable',
                                                                                       **self.map_args).opts(**self.map_opts)
            self.bm_map = self.ds_sel['Biomass'].hvplot(x='x', y='y',
                                       cmap=self.cbar_dict['Biomass']['cmap'],
                                       clim=self.cbar_dict['Biomass']['clim'],
                                       colorbar=False,
                                       **self.map_args).opts(**self.map_opts)
            self.thresh_map = da_thresh.hvplot(x='x', y='y',
                                               cmap=self.cbar_dict['Biomass probability']['cmap'],
                                               clim=self.cbar_dict['Biomass probability']['clim'], 
                                               colorbar=False,
                                               **self.map_args).options(color_levels=5).opts(**self.map_opts)

            self.ndvi_map = self.ds_sel['NDVI'].hvplot(x='x', y='y',
                                           cmap=self.cbar_dict['Greenness (NDVI)']['cmap'],
                                           clim=self.cbar_dict['Greenness (NDVI)']['clim'],
                                           colorbar=False,
                                           **self.map_args).opts(**self.map_opts)

            #self.all_maps = pn.Tabs(
            #    ('Cover', self.cov_map),
            #    ('Biomass', self.bm_map),
            #    ('Biomass threshold', self.thresh_map), 
            #    ('Greenness (NDVI)', self.ndvi_map))

            if self.base_rng.x_range is not None:
                self.all_maps = pn.Pane(self.tiles.apply.opts(xlim=self.base_rng.param.x_range,
                                                        ylim=self.base_rng.param.y_range)  * 
                                        self.labels.apply.opts(xlim=self.base_rng.param.x_range,
                                                               ylim=self.base_rng.param.y_range) * 
                                        self.cov_map.apply.opts(visible=self.cov_viz.param.value, 
                                                                xlim=self.base_rng.param.x_range,
                                                                ylim=self.base_rng.param.y_range) *
                                        self.ndvi_map.apply.opts(visible=self.ndvi_viz.param.value, 
                                                                 xlim=self.base_rng.param.x_range,
                                                                 ylim=self.base_rng.param.y_range) *
                                        self.bm_map.apply.opts(visible=self.bm_viz.param.value, 
                                                               xlim=self.base_rng.param.x_range,
                                                               ylim=self.base_rng.param.y_range) *
                                        self.thresh_map.apply.opts(visible=self.thresh_viz.param.value, 
                                                                   xlim=self.base_rng.param.x_range,
                                                                   ylim=self.base_rng.param.y_range) *
                                        self.polys)
                                         
                        
                        

                #self.all_maps = self.cov_map.apply.opts(visible=self.cov_viz.param.value) * self.polys

                #self.poly_stream = streams.PolyDraw(source=self.polys, drag=True, num_objects=2,
                #                        show_vertices=True, styles=self.poly_opts)

                return self.all_maps
            else:
                print('test')
                return pn.Column(self.tiles * self.labels)
        else:
            return pn.Column(self.tiles * self.labels)

    def update_stats(self, data):
        if data is None:
            self.bm_gauge.value = 0
        elif len(data['xs']) == 0:
            self.bm_gauge.value = 0
        else:
            if len(data['xs'][0]) < 3:
                self.bm_gauge.value = 0
                self.cov_pie = {
                    'title': {
                        'text': "Cover",
                        'subtext': "Fractional vegetation cover (%)",
                        'left': "center"},
                    'tooltip': {'show': True,
                                'formatter': '{b}: {d} %'}, 
                    'series': [{
                        'type': 'pie',
                        'data': [{'value': 0, 'name': 'Litter'},
                                 {'value': 0, 'name': 'Bare ground'},
                                 {'value': 0, 'name': 'Green veg'},
                                 {'value': 0, 'name': 'Dry veg'}],
                        'color': ['#ee6666', '#91cc75', '#5470c6', '#fac858'],
                        'roseType': 'radius',
                        'radius': ["0%", "45%"],
                        'label': {
                            'edgeDistance': "1%",
                            'bleedMargin': 10,
                            'alignTo': "edge"},
                        'labelLine': {'length': 5}}]}
                
                self.thresh_bar = {
                    'title': {
                        'text': "Biomass threshold",
                        'left': "center"},
                    'tooltip': {},
                    'legend': {},
                    'grid': {'left': '20%'},
                    'xAxis': {
                        'type': 'category',
                        'name': 'Probability of biomass less than ' + str(self.thresh_picker.value) + 'kg/ha',
                        'nameLocation': 'middle',
                        'nameGap': 30,
                        'data': self.thresh_labels},
                    'yAxis': {'max': 1000,
                              'axisLabel': {'formatter': '{value} ha'}},
                    'series': [{
                        'name': "",
                        'type': "bar",
                        'data': [0 for x in self.thresh_labels],
                        'colorBy': "data"}]}
            else:
                self.bm_gauge.loading = True
                self.bm_echart.loading = True
                self.cov_echart.loading = True
                self.thresh_echart.loading = True
                self.ts_bm_echart.loading = True
                self.ts_cov_echart.loading = True
                self.ts_ndvi_echart.loading = True
                polys_tmp = gpd.GeoDataFrame(data=data)
                polys_tmp.set_geometry(polys_tmp.apply(lambda row: Polygon(zip(row['xs'], row['ys'])), axis=1), inplace=True)
                polys_tmp.set_crs(epsg='3857', inplace=True)
                polys_info = polys_tmp[['line_color', 'geometry']].reset_index(drop=True).reset_index().rename(columns={'index': 'id'})
                polys_mask_shp = [(row.geometry, row.id+1) for _, row in polys_info.iterrows()]
                polys_mask = shp2mask(shp=polys_mask_shp, 
                                     transform=self.bm_map[:].data['Biomass'].rio.transform(), 
                                     outshape=self.bm_map[:].data['Biomass'].shape, 
                                     xr_object=self.bm_map[:].data['Biomass'])
                poly_mask_tmp = polys_mask.where(polys_mask == 1, drop=True)
                bm_dat_tmp = self.bm_map[:].data.sel(x=poly_mask_tmp['x'], y=poly_mask_tmp['y'], method='nearest', tolerance=30)
                ds_poly_tmp = self.ds_ts.sel(x=poly_mask_tmp['x'], y=poly_mask_tmp['y'], method='nearest', tolerance=30)#.compute()
                if not bm_dat_tmp.isnull().all()['Biomass']:
                    bm_hist_dat = bm_dat_tmp.groupby_bins('Biomass', bins=self.bm_plotrange, include_lowest=True,
                                    labels=self.bm_plotrange[1:]).count() / bm_dat_tmp.count() * 100
                    bm_hist_dat = bm_hist_dat.fillna(0)
                    self.bm_gauge.value = int(bm_dat_tmp.mean()['Biomass'].values)
                    self.bm_gauge.loading = False

                    thresh_dat_tmp = self.thresh_map[:].data.sel(x=poly_mask_tmp['x'], y=poly_mask_tmp['y'], method='nearest', tolerance=30)
                    thresh_hist_dat = thresh_dat_tmp.groupby_bins('Threshold', bins=self.thresh_plotrange, include_lowest=True,
                                    labels=self.thresh_labels).count() * 0.90
                    thresh_hist_dat = thresh_hist_dat.fillna(0)

                    da_cov_tmp = self.cov_map[:].data.sel(x=poly_mask_tmp['x'], y=poly_mask_tmp['y'], method='nearest', tolerance=30)
                    cov_factors = [k for k in self.cov_map[:].data.keys() if k not in ['y', 'x']]
                    cov_labels = [self.cov_dict[i] for i in cov_factors]
                    cov_labels.append('Litter')
                    cov_vals = [int(round(float(da_cov_tmp[f].mean())*100, 0)) for f in cov_factors]
                    cov_vals.append(int(round(100 - np.sum(cov_vals), 0)))
                    
                    self.bm_hist = {
                        'title': {
                            'text': 'Biomass variability',
                            'subtext': str(bm_hist_dat['Biomass'].cumsum()[
                                    bm_hist_dat['Biomass_bins'] == self.thresh_picker.value].astype('int').values[0]) +
                            '% of the area is less than the threshold of ' + 
                            str(self.thresh_picker.value) + ' kg/ha',
                            'left': "center"},
                        'tooltip': {},
                        'legend': {},
                        'grid': {'show': False},
                        'xAxis': {
                            'type': 'value',
                            'name': 'Biomass (kg/ha)',
                            'nameLocation': 'middle',
                            'nameGap': 30,
                            'splitLine': {
                                      'show': False}},
                        'yAxis': {'max': round(max(bm_hist_dat.Biomass.values)*1.35, 0),
                                  'axisLabel': {'formatter': '{value} %'},
                                  'splitLine': {
                                      'show': False}},
                        'series': [{
                            'name': "",
                            'type': "bar",
                            'data': [{'value': [bm_hist_dat.Biomass_bins.values[idx], x], 
                                      'itemStyle': {'color': self.bm_colors[idx]}} for idx, x in enumerate(
                                np.round(bm_hist_dat.Biomass.values, 1))],
                            'colorBy': "data",
                            'markLine': {
                                'silent': True,
                                'data': [
                                    [{'coord': [self.thresh_picker.value, 0],
                                      'lineStyle': {
                                          'color': 'black'}}, 
                                     {'coord': [self.thresh_picker.value, round(max(bm_hist_dat.Biomass.values)*1.15, 0)],
                                      'symbol': 'none'}]],
                                'lineStyle': {'color': 'black'},
                                'label': {'formatter': str(self.thresh_picker.value) + ' kg/ha', 
                                          'distance': 10,
                                          'color': 'black'}}}]}
                    self.bm_echart.object = self.bm_hist
                    self.bm_echart.loading = False
                    
                    self.thresh_bar = {
                        'title': {
                            'text': "Biomass threshold",
                            'subtext': 'Area in each category (' + str(int(bm_dat_tmp.count()['Biomass']*0.90)) + 'ha total)', 
                            'left': "center"},
                        'tooltip': {},
                        'legend': {},
                        'grid': {'left': '20%'},
                        'xAxis': {
                            'type': 'category',
                            'name': 'Probability of biomass less than ' + str(self.thresh_picker.value) + 'kg/ha',
                            'nameLocation': 'middle',
                            'nameGap': 30,
                            'data': thresh_hist_dat.Threshold_bins.values},
                        'yAxis': {'max': round(max(thresh_hist_dat.Threshold.values)*1.10, -1),
                                  'axisLabel': {'formatter': '{value} ha'}},
                        'series': [{
                            'name': "",
                            'type': "bar",
                            'data': [{'value': x, 'itemStyle': {'color': self.thresh_colors[idx]}} for idx, x in enumerate(
                                np.round(thresh_hist_dat.Threshold.values, -1))],
                            'colorBy': "data"
                        }],
                    }
                    self.thresh_echart.object = self.thresh_bar
                    self.thresh_echart.loading = False

                    self.cov_pie = {
                        'title': {
                            'text': "Cover",
                            'subtext': "Fractional vegetation cover (%)",
                            'left': "center"},
                        'tooltip': {'show': True,
                                    'formatter': '{b}: {d} %'}, 
                        'series': [{
                            'type': 'pie',
                            'data': [{'value': cov_vals[0], 'name': cov_labels[0]},
                                     {'value': cov_vals[1], 'name': cov_labels[1]},
                                     {'value': cov_vals[2], 'name': cov_labels[2]},
                                     {'value': cov_vals[3], 'name': cov_labels[3]}],
                            'color': ['#ee6666', '#91cc75', '#5470c6', '#fac858'],
                            'roseType': 'radius',
                            'radius': ["0%", "45%"],
                            'label': {
                                'edgeDistance': "1%",
                                'bleedMargin': 10,
                                'alignTo': "edge"},
                            'labelLine': {'length': 5}}]}

                    self.cov_echart.object = self.cov_pie
                    self.cov_echart.loading = False
                    
                    self.ts_bm = {
                        'title': [{'left': 'center', 'text': 'Biomass'}],
                        'legend': {'orient': 'vertical',
                                   'right': 10,
                                   'top': 'center'
                                  },
                        'tooltip': {'trigger': 'axis'},
                        'xAxis': [{'data': self.ds['date'].astype('str').values}],
                        'yAxis': [{}],
                        'series': [
                                  {'name': 'Pasture',
                                    'type': 'line',
                                    'showSymbol': False,
                                    'data': ds_poly_tmp.mean(['y', 'x'])['Biomass'].astype('int').values,
                                    'itemStyle': {'color': '#1b9e77'}}]}
                    self.ts_bm_echart.object = self.ts_bm
                    self.ts_bm_echart.loading = False
                    
                    self.ts_ndvi = {
                        'title': [{'left': 'center', 'text': 'Greenness (NDVI)'}],
                        'tooltip': {'trigger': 'axis'},
                        'xAxis': [{'data': self.ds['date'].astype('str').values}],
                        'yAxis': [{}],
                        'series': [
                                  {'name': 'Pasture',
                                    'type': 'line',
                                    'showSymbol': False,
                                    'data': ds_poly_tmp.mean(['y', 'x'])['NDVI'].values,
                                    'itemStyle': {'color': '#1b9e77'}}]}
                    self.ts_ndvi_echart.object = self.ts_ndvi
                    self.ts_ndvi_echart.loading = False
                    
                    self.ts_cov = {
                        'title': [{'left': 'center', 'text': 'Cover (fractional)'}],
                        'tooltip': {'trigger': 'axis'},
                        'xAxis': [{'data': ds['date'].astype('str').values}],
                        'yAxis': [{}],
                        'series': [{'name': 'Bare',
                                    'type': 'line',
                                    'showSymbol': False,
                                    'data': ds_poly_tmp.mean(['y', 'x'])['BARE'].values,
                                    'itemStyle': {'color': 'rgb(0, 0, 200)'},
                                    'lineStyle': {'type': 'solid'}},
                                  {'name': 'Litter',
                                    'type': 'line',
                                    'showSymbol': False,
                                    'data': ds_poly_tmp.mean(['y', 'x'])['LITT'].values,
                                    'itemStyle': {'color': 'rgb(200, 200, 10)'},
                                    'lineStyle': {'type': 'solid'}},
                                  {'name': 'Dry',
                                    'type': 'line',
                                    'showSymbol': False,
                                    'data': ds_poly_tmp.mean(['y', 'x'])['SD'].values,
                                    'itemStyle': {'color': 'rgb(200, 0, 0)'},
                                    'lineStyle': {'type': 'solid'}},
                                  {'name': 'Green',
                                    'type': 'line',
                                    'showSymbol': False,
                                    'data': ds_poly_tmp.mean(['y', 'x'])['GREEN'].values,
                                    'itemStyle': {'color': 'rgb(0, 175, 0)'}, 
                                    'lineStyle': {'type': 'solid'}}]}
                    self.ts_cov_echart.object = self.ts_cov
                    self.ts_cov_echart.loading = False

                    
    
    @pm.depends('thresh_picker.value', watch=True)
    def trigger_thresh(self):
        if not self.poly_stream.data is None:
            if len(self.poly_stream.data['xs']) > 0:
                self.update_stats(data=self.poly_stream.data)
    
    @pm.depends('viz_sel.value')
    def update_colorbar(self):
        if self.viz_sel.value == 'Basemap':
            return pn.Spacer(height=20, width=self.control_width)
        elif self.viz_sel.value == 'Biomass probability':
            return self.make_colorbar(self.viz_sel.value).options(color_levels=5)
        else:
            return self.make_colorbar(self.viz_sel.value)
    
    def _create_view(self):
        layout = pn.Column(pn.Row(pn.Column(self.year_picker, 
                                            self.date, 
                                            self.controls,
                                            width=int(self.control_width*1.2)), 
                                  pn.Column(self.create_maps),
                                 pn.Tabs(('Single-date', pn.Column(self.bm_gauge,
                                                                   self.bm_echart, 
                                                                   self.thresh_echart, 
                                                                   self.cov_echart, scroll=True,
                                           max_height=650, min_width=int(self.stats_width*1.1))),
                                         ('Time-series', pn.Column(self.ts_cov_echart,
                                                                   self.ts_bm_echart,
                                                                   self.ts_ndvi_echart)))))
        return layout
                        

In [ ]:
app = getData()
app.view.servable()
#pn.serve(app.view.servable())

In [ ]:
#app_served.stop()